<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/tiny_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiny-ImageNet
It contains 200 classes from full imagenet dataset.<br>
However, file structures are not for train and validation images. Train images are separated with a class-wise folder name where all validation images are a single folder. We need an additional propcessing to rearrange validation images into class-wise folders.

https://www.kaggle.com/c/tiny-imagenet

"MicroImageNet classification challenge is similar to the classification challenge in the full ImageNet ILSVRC. MicroImageNet contains 200 classes for training. Each class has 500 images. The test set contains 10,000 images. All images are 64x64 colored ones.

Your objective is to classify the 10,000 test set as accurately as possible."



*   train.images.zip - the training set (images distributed into class labeled folders)

*   test.zip - the unlabeled 10,000 test images

*   sample.txt - a sample submission file in the correct format (but needs to have 10,001 lines. One line per image in addition to the first header line)

*   wnids.txt - list of the used ids from the original full set of ImageNet

*   words.txt - description of all ids of ImageNet

Donwload & unzip

In [1]:
! wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
! unzip -q tiny-imagenet-200.zip

--2022-04-29 22:40:57--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  18.5MB/s    in 17s     

2022-04-29 22:41:15 (13.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



Training CNN model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--train_batch', default=512, type=int, help='batch size')
parser.add_argument('--valid_batch', default=1024, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=200, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

Training

In [ ]:
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset
import glob
from PIL import Image

seed_everything()
VALID_DIR = 'tiny-imagenet-200/val'
TRAIN_DIR = 'tiny-imagenet-200/train'

class TinyImageNetDataset(Dataset):
    def __init__(self, root, id, transform=None, train=False):
        self.transform = transform
        self.id_dict = id
        self.train = train
        if self.train:
            self.filenames = glob.glob(os.path.join(root, "train/*/*/*.JPEG")) 
        else:
            self.filenames = glob.glob(os.path.join(root,"val/images/*.JPEG"))
            self.cls_dic = {}
            for i, line in enumerate(open(os.path.join(root,'val/val_annotations.txt'), 'r')):
                a = line.split('\t')
                img, cls_id = a[0],a[1]
                self.cls_dic[img] = id[cls_id]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = Image.open(img_path).convert('RGB')

        if self.train:
            label = self.id_dict[img_path.split('/')[4]]
        else:
            label = self.cls_dic[img_path.split('/')[-1]]

        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([transforms.Resize(64), transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )
transform_test = transforms.Compose([transforms.Resize(64), transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )

id_dict = {}
for i, line in enumerate(open('/content/tiny-imagenet-200/wnids.txt', 'r')):
  id_dict[line.replace('\n', '')] = i

root = '/content/tiny-imagenet-200/'
train_dataset = TinyImageNetDataset(root=root, id=id_dict, transform=transform_train, train=True)
test_dataset = TinyImageNetDataset(root=root, id=id_dict, transform=transform_test, train=False)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.train_batch, shuffle=True,
                                          num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.valid_batch, shuffle=False, 
                                         num_workers=2, pin_memory=True)
print('sample size- Train:%d, Validation:%d'%(len(train_dataset), len(test_dataset)))


model = models.resnet50(pretrained=True).to(device)
model.fc = torch.nn.Linear(model.fc.in_features, args.num_classes)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
best_epoch, best_acc = 0, 0
for epoch in range(args.num_epoch):
    for epoch in range(args.num_epoch):
        if epoch != 0 and epoch < 100 and epoch % 30 == 0:
            for param in optimizer.param_groups:
                param['lr'] = param['lr'] / 10 
        train(model, trainloader, criterion, optimizer)
        accuracy = test(model, testloader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_tiny_imagenet.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

sample size- Train:100000, Validation:10000
epoch: 0  acc: 0.3851  best epoch: 0  best acc: 0.3851
epoch: 1  acc: 0.4043  best epoch: 1  best acc: 0.4043
epoch: 2  acc: 0.4120  best epoch: 2  best acc: 0.4120
epoch: 3  acc: 0.4340  best epoch: 3  best acc: 0.4340
epoch: 4  acc: 0.4574  best epoch: 4  best acc: 0.4574
epoch: 5  acc: 0.4454  best epoch: 4  best acc: 0.4574
epoch: 6  acc: 0.4485  best epoch: 4  best acc: 0.4574
epoch: 7  acc: 0.4212  best epoch: 4  best acc: 0.4574
epoch: 8  acc: 0.4346  best epoch: 4  best acc: 0.4574
epoch: 9  acc: 0.4547  best epoch: 4  best acc: 0.4574
epoch: 10  acc: 0.4495  best epoch: 4  best acc: 0.4574
epoch: 11  acc: 0.4484  best epoch: 4  best acc: 0.4574
epoch: 12  acc: 0.4227  best epoch: 4  best acc: 0.4574
epoch: 13  acc: 0.4337  best epoch: 4  best acc: 0.4574
epoch: 14  acc: 0.4308  best epoch: 4  best acc: 0.4574
epoch: 15  acc: 0.4370  best epoch: 4  best acc: 0.4574
epoch: 16  acc: 0.4510  best epoch: 4  best acc: 0.4574


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

epoch: 17  acc: 0.4492  best epoch: 4  best acc: 0.4574


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

epoch: 18  acc: 0.4170  best epoch: 4  best acc: 0.4574


...

epoch: 29  acc: 0.4142  best epoch: 4  best acc: 0.4574


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

epoch: 30  acc: 0.5422  best epoch: 30  best acc: 0.5422


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62b28680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

epoch: 31  acc: 0.5448  best epoch: 31  best acc: 0.5448


In [ ]:
epoch: 31  acc: 0.5448  best epoch: 31  best acc: 0.5448